In [10]:
import requests
import random
import plotly.graph_objs as go
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

app_id = "de88bad7" 
app_key = "794c067d87e1a94649aa25a00ed3c2a2" 
daily_calorie_goal = 1800

user_feedback = []

def create_filters(health_condition, ingredients_like, ingredients_dislike):
    filters = {
        "health": [health_condition] if health_condition else [],  # Add health filter if provided
        "exclude": ingredients_dislike.split(",") if ingredients_dislike else []  # Exclude disliked ingredients
    }
    return filters

def get_meal(meal_type, min_calories, max_calories, filters):
    url = f"https://api.edamam.com/search?q={meal_type}&app_id={app_id}&app_key={app_key}&calories={min_calories}-{max_calories}&from=0&to=10"
    
    if filters.get('health'):
        health_filters = '&health=' + '&health='.join(filters['health'])
        url += health_filters
    if filters.get('exclude'):
        exclude_filters = '&excluded=' + '&excluded='.join(filters['exclude'])
        url += exclude_filters

    try:
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json()
        if data.get('hits'):
            return data['hits']
        return []
    except requests.exceptions.RequestException as e:
        print(f"Error fetching meal: {e}")
        return []

def generate_day_plan(filters):
    breakfast_calories = daily_calorie_goal * 0.25
    lunch_calories = daily_calorie_goal * 0.35
    dinner_calories = daily_calorie_goal * 0.35
    snack_calories = daily_calorie_goal * 0.05

    breakfast_recipes = get_meal('breakfast', int(breakfast_calories * 0.8), int(breakfast_calories * 1.2), filters)
    lunch_recipes = get_meal('lunch', int(lunch_calories * 0.8), int(lunch_calories * 1.2), filters)
    dinner_recipes = get_meal('dinner', int(dinner_calories * 0.8), int(dinner_calories * 1.2), filters)
    snack_recipes = get_meal('snack', int(snack_calories * 0.8), int(snack_calories * 1.2), filters)

    def select_random_recipe(recipes):
        if recipes:
            return random.choice(recipes)['recipe']
        return None

    def collect_nutrition(recipe):
        if recipe:
            try:
                return {
                    "calories": recipe.get('calories', 0),
                    "protein": recipe['totalNutrients'].get('PROCNT', {}).get('quantity', 0),
                    "fat": recipe['totalNutrients'].get('FAT', {}).get('quantity', 0),
                    "carbs": recipe['totalNutrients'].get('CHOCDF', {}).get('quantity', 0)
                }
            except KeyError:
                print(f"Missing nutrient info for recipe: {recipe['label']}")
                return {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}
        return {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}

    meals = {
        "Breakfast": select_random_recipe(breakfast_recipes),
        "Lunch": select_random_recipe(lunch_recipes),
        "Dinner": select_random_recipe(dinner_recipes),
        "Snack": select_random_recipe(snack_recipes)
    }

    nutrition = {
        "calories": 0,
        "protein": 0,
        "fat": 0,
        "carbs": 0
    }

    for meal in meals.values():
        meal_nutrition = collect_nutrition(meal)
        nutrition['calories'] += meal_nutrition['calories']
        nutrition['protein'] += meal_nutrition['protein']
        nutrition['fat'] += meal_nutrition['fat']
        nutrition['carbs'] += meal_nutrition['carbs']

    return meals, nutrition

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        health_condition = request.form['health_condition']
        ingredients_like = request.form['ingredients_like']
        ingredients_dislike = request.form['ingredients_dislike']

        filters = create_filters(health_condition, ingredients_like, ingredients_dislike)

        monthly_plan = {}
        monthly_nutrition = {
            "calories": [],
            "protein": [],
            "fat": [],
            "carbs": []
        }

        for day in range(1, 31):
            day_plan, nutrition = generate_day_plan(filters)
            monthly_plan[day] = day_plan
            monthly_nutrition["calories"].append(nutrition["calories"])
            monthly_nutrition["protein"].append(nutrition["protein"])
            monthly_nutrition["fat"].append(nutrition["fat"])
            monthly_nutrition["carbs"].append(nutrition["carbs"])

        days = list(range(1, 31))

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=days, y=monthly_nutrition["calories"], mode='lines+markers', name='Calories'))
        fig.add_trace(go.Scatter(x=days, y=monthly_nutrition["protein"], mode='lines+markers', name='Protein'))
        fig.add_trace(go.Scatter(x=days, y=monthly_nutrition["fat"], mode='lines+markers', name='Fat'))
        fig.add_trace(go.Scatter(x=days, y=monthly_nutrition["carbs"], mode='lines+markers', name='Carbs'))

        fig.update_layout(title='Monthly Nutritional Analysis', xaxis_title='Day', yaxis_title='Nutrient Amount')

        plot_div = fig.to_html(full_html=False)

        return render_template('month_result.html', meal_plan=monthly_plan, plot_div=plot_div)

    return render_template('index.html')

@app.route('/feedback', methods=['GET', 'POST'])
def feedback():
    if request.method == 'POST':
        rating = request.form['rating']
        comments = request.form['comments']

        feedback_entry = {"rating": rating, "comments": comments}
        user_feedback.append(feedback_entry)

        return redirect(url_for('feedback'))

    return render_template('feedback.html')

@app.route('/show_feedback')
def show_feedback():
    return render_template('feedback.html', feedback_list=user_feedback)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Sep/2024 17:35:58] "GET / HTTP/1.1" 200 -


Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=breakfast&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=360-540&from=0&to=10
Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=dinner&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=504-756&from=0&to=10
Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=snack&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=72-108&from=0&to=10
Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=breakfast&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=360-540&from=0&to=10
Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=lunch&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=504-756&from=0&to=10
Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=dinner&app_id=de88bad7&app_key=794c067d87

127.0.0.1 - - [21/Sep/2024 17:37:36] "POST / HTTP/1.1" 200 -


Error fetching meal: 429 Client Error:  for url: https://api.edamam.com/search?q=snack&app_id=de88bad7&app_key=794c067d87e1a94649aa25a00ed3c2a2&calories=72-108&from=0&to=10


127.0.0.1 - - [21/Sep/2024 17:38:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2024 17:40:16] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2024 17:40:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2024 17:42:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2024 17:42:30] "GET /feedback HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2024 17:42:33] "GET / HTTP/1.1" 200 -
